# Web Scraping SSCASN BKN: From Interactive Elements to Meaningful Data

Okay, so here’s the deal. I’m using this notebook to scrape data from the SSCASN BKN website. Why? Because I’m trying to figure out which job formations have a higher chance of getting me through the CPNS 2025 selection process. Instead of manually shifting through pages and pages of information, Why not let Python do the boring work?

This project is super personal and I’m basically building my own cheat sheet to make smarter decisions during the application process. Plus, I get to brush up on my web scraping skills while I’m at it. Win-win, right?

What’s the Plan?

The SSCASN website has everything I need, but it’s not exactly user-friendly for data nerds like me. Especially the filter that only allow us to select one-selection per load. Hence, for the python scrape automation challenge is:

- Dropdown menus that dynamically load options.
- Pagination, because of course, all the good stuff is spread across multiple pages.
- Dynamic tables that make scraping a bit tricky.
- The job description of each formation is described in different link. How to gather all that data needed??

The goal? 

Automate the whole process so I can get clean data in one go. Then let's analyze the data retrieved

In [1]:
import pandas as pd
import math
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [50]:
# This is path to the chromedriver. You can adjust as you need
path_chromedriver = os.path.join(os.getcwd(), "chromedriver-win64", "chromedriver.exe")

chrome_options = Options()
service = Service(executable_path=path_chromedriver)

In [59]:
# Let's load the Driver
driver = webdriver.Chrome(service=service, options=chrome_options)
url = "https://sscasn.bkn.go.id/" # this is the website links
driver.get(url)

This function was initially created for development purposes to reset the web driver back to the homepage of the SSCASN website. It was particularly useful for troubleshooting or ensuring the code ran smoothly by restarting the scraping process from a clean state. 

Now, it is also used as a convenient way to return to the homepage during operations

In [4]:
def reset_scrap(): 
    """
    This function is created to reset the website into homepage.    
    """
    url = "https://sscasn.bkn.go.id/"
    driver.get(url)  
reset_scrap()

Here I limit the filters into my desired criteria, you may adjust it as you need 
(make sure the text matches exactly what already on the websites)

In [5]:
jenjang = "S-1/Sarjana"
pengadaan = "CPNS"

Since I want to perform multiple searchers, My plan is to loop through the prodi list and retrieve all the data for each prodi.

In [8]:
prodi_list = [
    "S-1 ILMU STATISTIK",
    "S-1 ILMU STATISTIKA",
    "S-1 KEPENDUDUKAN DAN STATISTIK",
    "S-1 KOMPUTASI STATISTIKA",
    "S-1 MATEMATIKA STATISTIKA",
    "S-1 STATISTIK",
    "S-1 STATISTIK (DATA ANALISIS)",
    "S-1 STATISTIK KEUANGAN",
    "S-1 STATISTIK KOMPUTASI",
    "S-1 STATISTIKA",
    "S-1 STATISTIKA BISNIS DAN INDUSTRI",
    "S-1 STATISTIKA DAN SAINS DATA",
    "S-1 STATISTIKA TERAPAN",
    "S-1/D-IV STATISTIK"
]

Here's is the data to interact with the filters.

In [6]:
all_data = [] #initiate base data

In [21]:
def interact_with_filter(jenjang,prodi):
    # Select the "Jenjang Pendidikan"
    jenjang_pendidikan = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="pencarian"]/div/div/form/div[1]/div[1]/div/div/div/input'))
    )
    jenjang_pendidikan.click()

    options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//li[contains(text(), 'S-1/Sarjana')]"))
    )

    for option in options:
        if option.text == jenjang:
            option.click() 
            break
    time.sleep(1)  # Give the website time to load

    # Select the "Prodi"
    prodi_path = '//*[@id="pencarian"]/div/div/form/div[1]/div[2]/div/div/div/input'
    prodi_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, prodi_path))
    )
    prodi_select.click()

    options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, f"//li[contains(text(),'{prodi}')]"))
    )

    for option in options:
        if option.text == prodi:
            option.click()
            break

    time.sleep(1)  # Give the website time to load

    # Select the "CPNS"
    pengadaan_path = '//*[@id="pencarian"]/div/div/form/div[1]/div[4]/div/div/div/input'
    pengadaan_select = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, pengadaan_path))
    )
    pengadaan_select.click()

    options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, f"//li[contains(text(),'{pengadaan}')]"))
    )

    for option in options:
        if option.text == pengadaan:
            option.click()

    time.sleep(1)  # Again, give the website time to load 

    # Finally, click the Search button
    cari_path = '//*[@id="pencarian"]/div/div/form/div[1]/div[5]/a'
    driver.find_element(By.XPATH, cari_path).click()

    time.sleep(5)  # Give the website time to load the tables

    # Let's check for how many pages there are
    total_formasi_path = driver.find_element(By.XPATH, '//*[@id="daftarFormasi"]/div[2]/div/div/div[3]/ul/li[1]')

    text_halaman = total_formasi_path.text
    total_formasi = text_halaman.split(': ')[1].split()[0]
    total_page = math.ceil(int(total_formasi) / 10)
    
    return total_page

This function defines how to interact with the pagination, specifically the '>' button. 

Why does the code look like this? After analying the html structure, I discovered that the button dynamically changes based on unique patterns. To handle this, first I create a base format and then loop through with criteria to match each logic.

In [8]:
def klik(index):
    """This function created to click the pagination next button"""
    
    button_xpath = f'//*[@id="daftarFormasi"]/div[2]/div/div/div[3]/ul/li[{index}]/button'
    button = driver.find_element(By.XPATH,button_xpath)
    button.click()

and here it is the main agenda, extracting the data. We will extract the data from dynamic table that changes based on pagination and filters. 

In [9]:
def extract_data():
    table_data = []
    table = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//table"))
    )

    rows = table.find_elements(By.XPATH, ".//tbody/tr")

    for row in rows:

        cells = row.find_elements(By.TAG_NAME, "td")
        row_data = {
            "Jabatan": cells[0].text,
            "Instansi": cells[1].text,
            "Unit Kerja": cells[2].text,
            "Formasi": cells[3].text,
            "(PPPK) Khusus disabilitas? (CPNS) Dapat Diisi Disabilitas?": cells[4].text,
            "Penghasilan (juta)": cells[5].text,
            "Jumlah Kebutuhan": cells[6].text,
            "Jumlah Lulus verifikasi": cells[7].text,
            "Link": cells[8].find_element(By.TAG_NAME, "a").get_attribute('href')
        }

        table_data.append(row_data)
    return table_data

In [67]:
def loop_data(start,total_page):
    """This function is created to loop the extracting progress"""
    page_index = 10  
    current_page = start

    for i in range(current_page,total_page-current_page):
        print(i)
        data = extract_data()
        all_data.extend(data)

        if current_page<=3:
            klik(page_index)

        elif current_page == total_page-3 or current_page == 4:
            page_index = 11
            klik(page_index)

        elif current_page >= total_page-2:
            page_index = 10
            klik(page_index)

        else:
            page_index = 12
            klik(page_index)
        
        current_page += 1
        if i%100==0: # adjust the website load
            time.sleep(15)
        else:
            continue
    reset_scrap()
    time.sleep(5)

In [ ]:
all_data = []
for prodi in prodi_list:
    pages = interact_with_filter(jenjang,prodi)
    loop_data(pages)

In [14]:
df = pd.DataFrame(all_data)
df.to_excel("extracted_data.xlsx", index=False) 

In [65]:
reset_scrap()

In [62]:
loop_data(549,966)

In [64]:
pd.DataFrame(all_data).to_excel("AKUNTANSI CPNS 2024.xlsx",index=False)

In [52]:
# request
all_data = []
loop_data(966)
pd.DataFrame(all_data).to_excel("AKUTANSI CPNS 2024.xlsx",index=False)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172


StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6867AFB05+28789]
	(No symbol) [0x00007FF6867186E0]
	(No symbol) [0x00007FF6865B592A]
	(No symbol) [0x00007FF6865BC48F]
	(No symbol) [0x00007FF6865BF4BF]
	(No symbol) [0x00007FF686650910]
	(No symbol) [0x00007FF68662F43A]
	(No symbol) [0x00007FF68664F654]
	(No symbol) [0x00007FF68662F1E3]
	(No symbol) [0x00007FF6865FA938]
	(No symbol) [0x00007FF6865FBAA1]
	GetHandleVerifier [0x00007FF686AE933D+3410093]
	GetHandleVerifier [0x00007FF686AFE7DD+3497293]
	GetHandleVerifier [0x00007FF686AF2A73+3448803]
	GetHandleVerifier [0x00007FF686877BBB+848171]
	(No symbol) [0x00007FF686723C3F]
	(No symbol) [0x00007FF68671F6E4]
	(No symbol) [0x00007FF68671F87D]
	(No